In [1]:
import torch
import pandas as pd
import pyaging as pya
import os

In [2]:
os.system("git clone https://github.com/Meyer-DH/AgingClock.git")

Cloning into 'AgingClock'...


0

In [3]:
df = pd.read_csv('AgingClock/Data/Predictor_Genes.csv')

df['feature'] = df['WormBaseID']
df['coefficient'] = df['ElasticNet_Coef']

In [4]:
features = df['feature'].tolist()

weights = torch.tensor(df['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([103.54631743289005])

In [5]:
model = pya.models.LinearModel(len(features))

model.linear.weight.data = weights
model.linear.bias.data = intercept

model

LinearModel(
  (linear): Linear(in_features=576, out_features=1, bias=True)
)

In [6]:
weights_dict = {
    'preprocessing': 'binarize', 
    'preprocessing_helper': None,
    'postprocessing': None,
    'postprocessing_helper': None,
    'features': features,
    'weight_dict': model.state_dict(),
}

metadata_dict = {
    'species': 'C elegans',
    'data_type': 'transcriptomics',
    'year': 2021,
    'citation': "Meyer, David H., and Björn Schumacher. \"BiT age: A transcriptome‐based aging clock near the theoretical limit of accuracy.\" Aging cell 20.3 (2021): e13320.",
    'doi': "https://doi.org/10.1111/acel.13320"
}

In [7]:
torch.save(weights_dict, '../weights/bitage.pt')
torch.save(metadata_dict, '../metadata/bitage.pt')

In [8]:
os.system("rm -r AgingClock")

0